<a href="https://colab.research.google.com/github/BrunoBmassa/Python-Actitivies/blob/main/preparacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [111]:
import pyspark
import gdown
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, date_format
from pyspark.ml.feature import StringIndexer, VectorAssembler, PCA, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import IntegerType
from datetime import date
from os.path import join

In [112]:
spark= SparkSession.builder.getOrCreate()


In [113]:
#Leia o arquivo ‘videos-tratados.snappy.parquet' no dataframe 'df_video'

df_video_url='https://drive.google.com/file/d/1nfM60gLkNeR_sMPQC8O3fvySdovEbrlt/view?usp=drive_link'
# Extração do arquivo pela URL
file_id = df_video_url.split('/')[-2]
# Download do arquivo
output_file = 'videos-tratados.snappy.parquet.csv'
gdown.download(id=file_id, output=output_file, quiet=False)
#Leitura do arquivo inferindo o esquema
df_video = spark.read.parquet(output_file)



Downloading...
From: https://drive.google.com/uc?id=1nfM60gLkNeR_sMPQC8O3fvySdovEbrlt
To: /content/videos-tratados.snappy.parquet.csv
100%|██████████| 1.99M/1.99M [00:00<00:00, 42.4MB/s]


In [115]:
#Adicione a coluna 'Month' com o valor do mês da coluna "Published At"

df_video = df_video.withColumn('Month', date_format(col('Published At'), 'MM').cast('int')) #Ajustado de 'mmmm' para 'mm' e colocado como inteiro

In [116]:
#Adicione a coluna "Keyword Index" com a transformação da coluna 'keyword' para valores numéricos

indexer = StringIndexer(inputCol='Keyword', outputCol='Keyword Index')
df_video = indexer.fit(df_video).transform(df_video)

In [117]:
#Crie um vetor chamado "Features" com os campos: "Likes", "Views", "Year", "Month", "Keyword Index" e transforme o dataframe df_video com o VectorAssembler, lembrando que o vetor só aceita campos do tipo numérico

df_video = df_video.withColumn("Year", df_video["Year"].cast(IntegerType())) ## Convertido para inteiro
df_video = df_video.na.drop()
montar_vetor = VectorAssembler(inputCols=['Likes', 'Views', 'Year', 'Month', 'Keyword Index'],
                               outputCol='Features')
df_video = montar_vetor.transform(df_video)



In [118]:
#Adicione a coluna "Features Normal" com os dados normalizados da coluna Features, lembrando que para normalizar a coluna não pode conter valores nulos

df_video = df_video.na.drop()

scaler= MinMaxScaler(inputCol='Features', outputCol='Features Normal')
modelo_scaler = scaler.fit(df_video)
df_video = modelo_scaler.transform(df_video)


In [119]:
#Adicione a coluna "Features PCA" com a redução de 5 características para 1, utilizando o modelo PCA

pca = PCA(k=1, inputCol='Features Normal', outputCol='Features PCA')
modelo_pca = pca.fit(df_video)
df_video = modelo_pca.transform(df_video)


In [120]:
#Separe o dataframe df_video em 2 conjuntos: 80% para treinamento e 20% para teste

df_treino, df_teste = df_video.randomSplit([0.8, 0.2], seed=42)


In [121]:
#Crie um modelo de regressão linear para estimar o valor do campo "Comments", utilizando a "Features Normal" e avalie o modelo

regressão_linear = LinearRegression(featuresCol='Features Normal', labelCol='Comments')
modelo_rl = regressão_linear.fit(df_treino)

avaliacao_rl = modelo_rl.evaluate(df_teste)
print("RMSE(Erro quadrático médio da raiz):", avaliacao_rl.rootMeanSquaredError)
print("R2(Coeficiente de determinação):", avaliacao_rl.r2)


RMSE(Erro quadrático médio da raiz): 15696.328970507317
R2(Coeficiente de determinação): 0.8423252962415371


In [122]:
#Salve o dataframe df_video como 'videos-preparados-parquet' no formato parquet

df_video.write.option('header','true').mode("overwrite").parquet('videos-preparados-parquet')
